In [1]:
import pandas as pd
import neptune

NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='

PROJECT_ID = 'revivemed/Survival-RCC'

In [9]:
df_file = '/Users/jonaheaton/Desktop/Survival-RCC (4).csv'
df = pd.read_csv(df_file)
# chosen_columns = ['desc_str','sweep_kwargs/batch_size','sweep_kwargs/encoder_weight']

In [37]:
start_columns = df.columns
start_columns = start_columns.drop(['Creation Time','Tags','Owner','Id'])
val_columns = []
test_columns = []
param_columns = []
for x in start_columns:
    if 'avg_training_run' in x:
        val_columns.append(x)
        new_x = x.replace('avg_training_run/','avg_testing_run/')
        if 'val__' in new_x:
            new_x = new_x.replace('val__','test__')
            test_columns.append(new_x)
        else:
            test_columns.append(new_x)
    else:
        param_columns.append(x)

chosen_columns = param_columns + val_columns + test_columns

In [38]:
project = neptune.init_project(
    project=PROJECT_ID, 
    mode="read-only",
)

runs_table_df = project.fetch_runs_table(
    query='(`avg_training_run/num_success`:int >= 5)',
    columns = chosen_columns,
).to_pandas()

if 'sys/id' not in param_columns:
    param_columns = ['sys/id']	+ param_columns

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/


In [39]:
runs_table_df = runs_table_df[param_columns + val_columns + test_columns]

In [40]:
runs_table_df

,sys/id,desc_str,sweep_kwargs/batch_size,sweep_kwargs/encoder_weight,sweep_kwargs/head_hidden_layers,sweep_kwargs/noise_factor,sweep_kwargs/use_rand_init,sweep_kwargs/num_epochs,sweep_kwargs/dropout_rate,sweep_kwargs/EVER-OS__weight,...,avg_testing_run/metrics/test__head_EVER OS__on_Both OS_Concordance Index,avg_testing_run/metrics/std_test__head_EVER OS__on_Both OS_Concordance Index,avg_testing_run/metrics/test__head_EVER OS__on_NIVO OS_Concordance Index,avg_testing_run/metrics/std_test__head_EVER OS__on_NIVO OS_Concordance Index,avg_testing_run/metrics/test__head_EVER OS__on_EVER OS_Concordance Index,avg_testing_run/metrics/std_test__head_EVER OS__on_EVER OS_Concordance Index,avg_testing_run/metrics/test__head_IMDC__on_IMDC BINARY_AUROC (micro),avg_testing_run/metrics/std_test__head_IMDC__on_IMDC BINARY_AUROC (micro),avg_testing_run/metrics/test__head_MSKCC__on_MSKCC BINARY_AUROC (micro),avg_testing_run/metrics/std_test__head_MSKCC__on_MSKCC BINARY_AUROC (micro)
0,SUR-2539,NIVO-OS AND EVER-OS,64.0,0.0,0,0.20,True,71,0.0,NaN,...,0.691475,0.003260,0.701233,0.006604,0.691253,0.008731,NaN,NaN,NaN,NaN
1,SUR-2538,EVER-OS,64.0,0.0,0,0.20,False,71,0.0,NaN,...,0.691257,0.010451,0.691354,0.012800,0.695225,0.018038,NaN,NaN,NaN,NaN
2,SUR-2537,EVER-OS,64.0,0.0,0,0.20,True,71,0.0,NaN,...,0.684929,0.003592,0.691682,0.004882,0.688649,0.004330,NaN,NaN,NaN,NaN
11,SUR-2527,NIVO-OS AND EVER-OS,NaN,NaN,1,0.25,False,87,NaN,10.0,...,0.708627,0.009583,0.716629,0.010115,0.698974,0.015192,NaN,NaN,NaN,NaN
12,SUR-2526,NIVO-OS AND EVER-OS,NaN,NaN,1,0.25,True,87,NaN,10.0,...,0.685769,0.006425,0.690640,0.008265,0.690415,0.011327,NaN,NaN,NaN,NaN
13,SUR-2525,EVER-OS,NaN,NaN,1,0.25,False,87,NaN,10.0,...,0.697191,0.013827,0.701471,0.012328,0.695643,0.018170,NaN,NaN,NaN,NaN
14,SUR-2524,EVER-OS,NaN,NaN,1,0.25,True,87,NaN,10.0,...,0.683359,0.005797,0.679098,0.008522,0.692691,0.010473,NaN,NaN,NaN,NaN
3,SUR-2536,NIVO-OS,64.0,0.0,0,0.20,False,71,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SUR-2535,NIVO-OS,64.0,0.0,0,0.20,True,71,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,SUR-2523,NIVO-OS,NaN,NaN,1,0.25,False,87,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
